In [1]:
import numpy as np

In [2]:
A = np.array([[2,-1,6],[0,1,-1],[-2,2,1]])

In [45]:
class naive_fp():

    def __init__(self, payoff_matrix, initial_i=0):
        """Variables named according to Ferguson's 'Game Theory',
        part II, chapter 4, page 46.
        
        Note: to ease indexing, the strategies begin at 0; i.e.,
        a 3x3 matrix will have strategies 0, 1, and 2 rather than
        1, 2, and 3. This also means that the index for the `round`
        starts at k = 0 (meaning only the inital play has occured).
        """
        
        # Round
        k = 0
        
        # Pure strategy selections for player I
        self.i = np.array([initial_i], dtype = int)
        # Incremental payoffs for player II
        self.s = np.array([[0,0,0]])
     
        # Pure strategy selection for player II
        self.j = [np.argmin(A[i[0]])]
        # Incremental payoffs for player I
        self.t = np.array([[0,0,0]])
        
        # Game values
        self.v_lower, self.v_upper = np.array([None]), np.array([None])
        
    def next_round(self):
               
        # Increment payoffs for player II
        self.s = np.append(self.s, A[i[k]] + self.s[k], axis=0)
        # Select best-response for player II
        self.j.append(np.argmin(self.s[k+1]))
        
        # Increment payoffs for player I
        self.t
        
    
    

In [67]:
i=[0]
s=np.array([[0,0,0]])
j=[np.argmin(A[i[0]])]
t=np.array([[0,0,0]])
k=0

In [68]:

s = np.append(s,[A[i[k]] + s[k]], axis=0)
np.argmin(s[k+1])
s

array([[ 0,  0,  0],
       [ 2, -1,  6]])

In [69]:
np.argmin([A[0]])

1

In [80]:
np.append(t, A[:,j[k]] + t[k], axis = 0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [81]:
t

array([[0, 0, 0]])

In [82]:
A[:,j[k]]

array([-1,  1,  2])

In [83]:
t[k]

array([0, 0, 0])